In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
import os
import pandas as pd
import pickle
import re

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/01_level1')
from level1_utils import run_level1, make_level1_design_matrix, get_events

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'

subnum = '01'
runnum='4'

In [3]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
# plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,fractalProb_ev,reward_ev,stim_ev
choice_st,1.000000,-0.796205,0.883857,0.415893,0.839958
cross_ev,-0.796205,1.000000,-0.549370,-0.711303,-0.789923
fractalProb_ev,0.883857,-0.549370,1.000000,0.027088,0.645227
reward_ev,0.415893,-0.711303,0.027088,1.000000,0.498400
stim_ev,0.839958,-0.789923,0.645227,0.498400,1.000000


In [4]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,cross_rt,fractalProb_ev,reward_ev,stim_ev,stim_rt
choice_st,1.000000,-0.723174,-0.205398,0.883857,0.415893,0.973426,-0.000522
cross_ev,-0.723174,1.000000,0.000112,-0.507547,-0.634884,-0.780758,-0.032182
cross_rt,-0.205398,0.000112,1.000000,-0.248806,-0.101781,-0.181907,-0.088604
fractalProb_ev,0.883857,-0.507547,-0.248806,1.000000,0.027088,0.761268,0.001233
reward_ev,0.415893,-0.634884,-0.101781,0.027088,1.000000,0.590911,-0.096463
stim_ev,0.973426,-0.780758,-0.181907,0.761268,0.590911,1.000000,0.000264
stim_rt,-0.000522,-0.032182,-0.088604,0.001233,-0.096463,0.000264,1.000000


In [5]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,fractalProb_ev,fractalProb_par,reward_ev,stim_ev
choice_st,1.000000,-0.796205,0.883857,0.664270,0.415893,0.839958
cross_ev,-0.796205,1.000000,-0.549370,-0.408095,-0.711303,-0.789923
fractalProb_ev,0.883857,-0.549370,1.000000,0.752487,0.027088,0.645227
fractalProb_par,0.664270,-0.408095,0.752487,1.000000,0.041661,0.418377
reward_ev,0.415893,-0.711303,0.027088,0.041661,1.000000,0.498400
stim_ev,0.839958,-0.789923,0.645227,0.418377,0.498400,1.000000


In [6]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,cross_rt,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,stim_rt
choice_st,1.000000,-0.723174,-0.205398,0.883857,0.664270,0.415893,0.973426,-0.000522
cross_ev,-0.723174,1.000000,0.000112,-0.507547,-0.350689,-0.634884,-0.780758,-0.032182
cross_rt,-0.205398,0.000112,1.000000,-0.248806,-0.211953,-0.101781,-0.181907,-0.088604
fractalProb_ev,0.883857,-0.507547,-0.248806,1.000000,0.752487,0.027088,0.761268,0.001233
fractalProb_par,0.664270,-0.350689,-0.211953,0.752487,1.000000,0.041661,0.571360,-0.133765
reward_ev,0.415893,-0.634884,-0.101781,0.027088,0.041661,1.000000,0.590911,-0.096463
stim_ev,0.973426,-0.780758,-0.181907,0.761268,0.571360,0.590911,1.000000,0.000264
stim_rt,-0.000522,-0.032182,-0.088604,0.001233,-0.133765,-0.096463,0.000264,1.000000


In [7]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiff_par
choice_st,1.000000,-0.796205,0.883857,0.664270,0.415893,0.839958,-0.023374
cross_ev,-0.796205,1.000000,-0.549370,-0.408095,-0.711303,-0.789923,0.019800
fractalProb_ev,0.883857,-0.549370,1.000000,0.752487,0.027088,0.645227,-0.018069
fractalProb_par,0.664270,-0.408095,0.752487,1.000000,0.041661,0.418377,0.031200
reward_ev,0.415893,-0.711303,0.027088,0.041661,1.000000,0.498400,-0.016335
stim_ev,0.839958,-0.789923,0.645227,0.418377,0.498400,1.000000,-0.024652
valDiff_par,-0.023374,0.019800,-0.018069,0.031200,-0.016335,-0.024652,1.000000


In [8]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,cross_rt,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,stim_rt,valDiff_par
choice_st,1.000000,-0.723174,-0.205398,0.883857,0.664270,0.415893,0.973426,-0.000522,-0.030833
cross_ev,-0.723174,1.000000,0.000112,-0.507547,-0.350689,-0.634884,-0.780758,-0.032182,0.055516
cross_rt,-0.205398,0.000112,1.000000,-0.248806,-0.211953,-0.101781,-0.181907,-0.088604,-0.010914
fractalProb_ev,0.883857,-0.507547,-0.248806,1.000000,0.752487,0.027088,0.761268,0.001233,-0.023458
fractalProb_par,0.664270,-0.350689,-0.211953,0.752487,1.000000,0.041661,0.571360,-0.133765,0.016154
reward_ev,0.415893,-0.634884,-0.101781,0.027088,0.041661,1.000000,0.590911,-0.096463,-0.022710
stim_ev,0.973426,-0.780758,-0.181907,0.761268,0.571360,0.590911,1.000000,0.000264,-0.034646
stim_rt,-0.000522,-0.032182,-0.088604,0.001233,-0.133765,-0.096463,0.000264,1.000000,0.002291
valDiff_par,-0.030833,0.055516,-0.010914,-0.023458,0.016154,-0.022710,-0.034646,0.002291,1.000000


In [9]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valChosen_par,valUnchosen_par
choice_st,1.000000,-0.796205,0.883857,0.664270,0.415893,0.839958,0.671858,0.620777
cross_ev,-0.796205,1.000000,-0.549370,-0.408095,-0.711303,-0.789923,-0.628781,-0.592433
fractalProb_ev,0.883857,-0.549370,1.000000,0.752487,0.027088,0.645227,0.514811,0.472571
fractalProb_par,0.664270,-0.408095,0.752487,1.000000,0.041661,0.418377,0.084545,0.092610
reward_ev,0.415893,-0.711303,0.027088,0.041661,1.000000,0.498400,0.397376,0.362029
stim_ev,0.839958,-0.789923,0.645227,0.418377,0.498400,1.000000,0.789053,0.793648
valChosen_par,0.671858,-0.628781,0.514811,0.084545,0.397376,0.789053,1.000000,0.882546
valUnchosen_par,0.620777,-0.592433,0.472571,0.092610,0.362029,0.793648,0.882546,1.000000


In [10]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,cross_rt,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,stim_rt,valChosen_par,valUnchosen_par
choice_st,1.000000,-0.723174,-0.205398,0.883857,0.664270,0.415893,0.973426,-0.000522,0.696472,0.656345
cross_ev,-0.723174,1.000000,0.000112,-0.507547,-0.350689,-0.634884,-0.780758,-0.032182,-0.576565,-0.549679
cross_rt,-0.205398,0.000112,1.000000,-0.248806,-0.211953,-0.101781,-0.181907,-0.088604,-0.132534,-0.138621
fractalProb_ev,0.883857,-0.507547,-0.248806,1.000000,0.752487,0.027088,0.761268,0.001233,0.543752,0.512681
fractalProb_par,0.664270,-0.350689,-0.211953,0.752487,1.000000,0.041661,0.571360,-0.133765,0.085166,0.082551
reward_ev,0.415893,-0.634884,-0.101781,0.027088,0.041661,1.000000,0.590911,-0.096463,0.409045,0.374802
stim_ev,0.973426,-0.780758,-0.181907,0.761268,0.571360,0.590911,1.000000,0.000264,0.715955,0.674776
stim_rt,-0.000522,-0.032182,-0.088604,0.001233,-0.133765,-0.096463,0.000264,1.000000,0.126775,0.225424
valChosen_par,0.696472,-0.576565,-0.132534,0.543752,0.085166,0.409045,0.715955,0.126775,1.000000,0.865324
valUnchosen_par,0.656345,-0.549679,-0.138621,0.512681,0.082551,0.374802,0.674776,0.225424,0.865324,1.000000


In [11]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiffFractal_par,valDiffLottery_par
choice_st,1.000000,-0.796205,0.883857,0.664270,0.415893,0.839958,-0.142409,0.001122
cross_ev,-0.796205,1.000000,-0.549370,-0.408095,-0.711303,-0.789923,0.141504,-0.004307
fractalProb_ev,0.883857,-0.549370,1.000000,0.752487,0.027088,0.645227,-0.107356,-0.001519
fractalProb_par,0.664270,-0.408095,0.752487,1.000000,0.041661,0.418377,-0.088551,0.029853
reward_ev,0.415893,-0.711303,0.027088,0.041661,1.000000,0.498400,-0.082548,-0.001794
stim_ev,0.839958,-0.789923,0.645227,0.418377,0.498400,1.000000,-0.198860,0.009642
valDiffFractal_par,-0.142409,0.141504,-0.107356,-0.088551,-0.082548,-0.198860,1.000000,0.044616
valDiffLottery_par,0.001122,-0.004307,-0.001519,0.029853,-0.001794,0.009642,0.044616,1.000000


In [12]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,cross_rt,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,stim_rt,valDiffFractal_par,valDiffLottery_par
choice_st,1.000000,-0.723174,-0.205398,0.883857,0.664270,0.415893,0.973426,-0.000522,-0.150980,-0.007594
cross_ev,-0.723174,1.000000,0.000112,-0.507547,-0.350689,-0.634884,-0.780758,-0.032182,0.200507,0.014872
cross_rt,-0.205398,0.000112,1.000000,-0.248806,-0.211953,-0.101781,-0.181907,-0.088604,-0.035605,-0.002440
fractalProb_ev,0.883857,-0.507547,-0.248806,1.000000,0.752487,0.027088,0.761268,0.001233,-0.118001,-0.007084
fractalProb_par,0.664270,-0.350689,-0.211953,0.752487,1.000000,0.041661,0.571360,-0.133765,-0.135279,0.010393
reward_ev,0.415893,-0.634884,-0.101781,0.027088,0.041661,1.000000,0.590911,-0.096463,-0.091677,-0.009599
stim_ev,0.973426,-0.780758,-0.181907,0.761268,0.571360,0.590911,1.000000,0.000264,-0.157528,-0.008938
stim_rt,-0.000522,-0.032182,-0.088604,0.001233,-0.133765,-0.096463,0.000264,1.000000,-0.004225,0.020377
valDiffFractal_par,-0.150980,0.200507,-0.035605,-0.118001,-0.135279,-0.091677,-0.157528,-0.004225,1.000000,0.098629
valDiffLottery_par,-0.007594,0.014872,-0.002440,-0.007084,0.010393,-0.009599,-0.008938,0.020377,0.098629,1.000000


In [13]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valChosenFractal_par,valChosenLottery_par,valUnchosenFractal_par,valUnchosenLottery_par
choice_st,1.000000,-0.796205,0.883857,0.664270,0.415893,0.839958,0.419998,0.793155,0.283055,0.710783
cross_ev,-0.796205,1.000000,-0.549370,-0.408095,-0.711303,-0.789923,-0.387208,-0.744353,-0.269706,-0.672629
fractalProb_ev,0.883857,-0.549370,1.000000,0.752487,0.027088,0.645227,0.324347,0.609504,0.219391,0.544033
fractalProb_par,0.664270,-0.408095,0.752487,1.000000,0.041661,0.418377,0.206611,0.370850,0.111617,0.404077
reward_ev,0.415893,-0.711303,0.027088,0.041661,1.000000,0.498400,0.250359,0.471123,0.165082,0.419410
stim_ev,0.839958,-0.789923,0.645227,0.418377,0.498400,1.000000,0.443149,0.932762,0.347893,0.866019
valChosenFractal_par,0.419998,-0.387208,0.324347,0.206611,0.250359,0.443149,1.000000,0.386017,0.111274,0.376896
valChosenLottery_par,0.793155,-0.744353,0.609504,0.370850,0.471123,0.932762,0.386017,1.000000,0.304468,0.834423
valUnchosenFractal_par,0.283055,-0.269706,0.219391,0.111617,0.165082,0.347893,0.111274,0.304468,1.000000,0.238069
valUnchosenLottery_par,0.710783,-0.672629,0.544033,0.404077,0.419410,0.866019,0.376896,0.834423,0.238069,1.000000


In [14]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choice_st,cross_ev,cross_rt,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,stim_rt,valChosenFractal_par,valChosenLottery_par,valUnchosenFractal_par,valUnchosenLottery_par
choice_st,1.000000,-0.723174,-0.205398,0.883857,0.664270,0.415893,0.973426,-0.000522,0.471699,0.899009,0.297535,0.805666
cross_ev,-0.723174,1.000000,0.000112,-0.507547,-0.350689,-0.634884,-0.780758,-0.032182,-0.382492,-0.716754,-0.247725,-0.644539
cross_rt,-0.205398,0.000112,1.000000,-0.248806,-0.211953,-0.101781,-0.181907,-0.088604,-0.046927,-0.170222,-0.037949,-0.150896
fractalProb_ev,0.883857,-0.507547,-0.248806,1.000000,0.752487,0.027088,0.761268,0.001233,0.367862,0.702855,0.234594,0.629434
fractalProb_par,0.664270,-0.350689,-0.211953,0.752487,1.000000,0.041661,0.571360,-0.133765,0.322728,0.490357,0.113580,0.529632
reward_ev,0.415893,-0.634884,-0.101781,0.027088,0.041661,1.000000,0.590911,-0.096463,0.300574,0.545030,0.172032,0.486020
stim_ev,0.973426,-0.780758,-0.181907,0.761268,0.571360,0.590911,1.000000,0.000264,0.485358,0.923179,0.305468,0.827220
stim_rt,-0.000522,-0.032182,-0.088604,0.001233,-0.133765,-0.096463,0.000264,1.000000,-0.169414,-0.001339,0.088200,0.018098
valChosenFractal_par,0.471699,-0.382492,-0.046927,0.367862,0.322728,0.300574,0.485358,-0.169414,1.000000,0.418606,0.057947,0.421508
valChosenLottery_par,0.899009,-0.716754,-0.170222,0.702855,0.490357,0.545030,0.923179,-0.001339,0.418606,1.000000,0.277647,0.797712


In [15]:
mnum = 'model1'
reg_rt = 1
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt")%(mnum, str(reg_rt)

run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt = reg_rt)

A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-1
***********************************************
***********************************************
Running GLM for sub-01 run-1
***********************************************
***********************************************
Saving GLM for sub-01 run-1
***********************************************
***********************************************
Running contrasts for sub-01 run-1
***********************************************
***********************************************
Done saving contrasts for sub-01 run-1
***********************************************
A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-2
***********************************************
***********************************************
Running GLM for sub-01 run-2


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


***********************************************
Saving design matrix for sub-01 run-4
***********************************************
***********************************************
Running GLM for sub-01 run-4
***********************************************
***********************************************
Saving GLM for sub-01 run-4
***********************************************
***********************************************
Running contrasts for sub-01 run-4
***********************************************
***********************************************
Done saving contrasts for sub-01 run-4
***********************************************
A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-5
***********************************************
***********************************************
Running GLM for sub-01 run-5
***********************************************
**********************

In [16]:
mnum = 'model2'
reg_rt = 0
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt")%(mnum, str(reg_rt)
                                                                                
run_level1(subnum, 'model2', data_path, behavior_path, out_path, regress_rt = 0)

A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-1
***********************************************
***********************************************
Running GLM for sub-01 run-1
***********************************************
***********************************************
Saving GLM for sub-01 run-1
***********************************************
***********************************************
Running contrasts for sub-01 run-1
***********************************************
***********************************************
Done saving contrasts for sub-01 run-1
***********************************************
A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-2
***********************************************
***********************************************
Running GLM for sub-01 run-2


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


***********************************************
Saving design matrix for sub-01 run-4
***********************************************
***********************************************
Running GLM for sub-01 run-4
***********************************************
***********************************************
Saving GLM for sub-01 run-4
***********************************************
***********************************************
Running contrasts for sub-01 run-4
***********************************************
***********************************************
Done saving contrasts for sub-01 run-4
***********************************************
A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-5
***********************************************
***********************************************
Running GLM for sub-01 run-5
***********************************************
**********************

Function to get average correlation in design matrices

In [ ]:
mnum = 'model1'
reg_rt = '1'
in_path = 
des_mats = glob.glob()